In [1]:
!pip install ISR

     |████████████████████████████████| 86.3MB 55kB/s 
     |████████████████████████████████| 450kB 39.0MB/s 
     |████████████████████████████████| 3.8MB 35.9MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=3a73cf220a9495de8d0baa391e4708e314c0da2a8f90ea20b36384d702e677e9
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: tensorboard 2.2.2
    Uninstalling tensorboard-2.2.2:
      Successfully uninstalled tensorboard-2.2.2
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: t

In [2]:
!wget http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_LR_bicubic_X2.zip
!wget http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_valid_LR_bicubic_X2.zip
!wget http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_HR.zip
!wget http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_valid_HR.zip


--2020-07-04 05:54:19--  http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_LR_bicubic_X2.zip
Resolving data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)... 129.132.52.162
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.162|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_LR_bicubic_X2.zip [following]
--2020-07-04 05:54:20--  https://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_LR_bicubic_X2.zip
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.162|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 925390592 (883M) [application/zip]
Saving to: ‘DIV2K_train_LR_bicubic_X2.zip’

DIV2K_train_LR_bicu 100%[===================>] 882.52M  8.16MB/s    in 1m 51s  

2020-07-04 05:56:12 (7.96 MB/s) - ‘DIV2K_train_LR_bicubic_X2.zip’ saved [925390592/925390592]

--2020-07-04 05:56:14--  http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_valid_LR_bicubic_X

In [3]:
!mkdir div2k
!unzip -q DIV2K_valid_LR_bicubic_X2.zip -d div2k
!unzip -q DIV2K_train_LR_bicubic_X2.zip -d div2k
!unzip -q DIV2K_train_HR.zip -d div2k
!unzip -q DIV2K_valid_HR.zip -d div2k


In [4]:
from ISR.models import RRDN
from ISR.models import Discriminator
from ISR.models import Cut_VGG19


In [5]:
lr_train_patch_size = 50
layers_to_extract = [5, 9]
scale = 2
hr_train_patch_size = lr_train_patch_size * scale

rrdn  = RRDN(arch_params={'C':4, 'D':3, 'G':64, 'G0':64, 'T':10, 'x':scale}, patch_size=lr_train_patch_size)
f_ext = Cut_VGG19(patch_size=hr_train_patch_size, layers_to_extract=layers_to_extract)
discr = Discriminator(patch_size=hr_train_patch_size, kernel_size=3)


80142336/80134624 [==============================] - 7s 0us/step


In [6]:
from ISR.train import Trainer
loss_weights = {
  'generator': 0.0,
  'feature_extractor': 0.0833,
  'discriminator': 0.01
}
losses = {
  'generator': 'mae',
  'feature_extractor': 'mse',
  'discriminator': 'binary_crossentropy'
} 

log_dirs = {'logs': './logs', 'weights': './weights'}

learning_rate = {'initial_value': 0.0004, 'decay_factor': 0.5, 'decay_frequency': 30}

flatness = {'min': 0.0, 'max': 0.15, 'increase': 0.01, 'increase_frequency': 5}

trainer = Trainer(
    generator=rrdn,
    discriminator=discr,
    feature_extractor=f_ext,
    lr_train_dir='div2k/DIV2K_train_LR_bicubic/X2/',
    hr_train_dir='div2k/DIV2K_train_HR/',
    lr_valid_dir='div2k/DIV2K_train_LR_bicubic/X2/',
    hr_valid_dir='div2k/DIV2K_train_HR/',
    loss_weights=loss_weights,
    learning_rate=learning_rate,
    flatness=flatness,
    dataname='div2k',
    log_dirs=log_dirs,
    weights_generator=None,
    weights_discriminator=None,
    n_validation=50,
)

In [7]:
trainer.train(
    epochs=2,
    steps_per_epoch=20,
    batch_size=4,
    monitored_metrics={'val_generator_PSNR_Y': 'max'}
)



Training details:
  training_parameters: 
    metrics: {'generator': <function PSNR_Y at 0x7f6c18ac6268>}
    losses: {'generator': 'mae', 'discriminator': 'binary_crossentropy', 'feature_extractor': 'mse'}
    adam_optimizer: {'beta1': 0.9, 'beta2': 0.999, 'epsilon': None}
    learning_rate: {'initial_value': 0.0004, 'decay_factor': 0.5, 'decay_frequency': 30}
    flatness: {'min': 0.0, 'max': 0.15, 'increase': 0.01, 'increase_frequency': 5}
    n_validation: 50
    dataname: div2k
    fallback_save_every_n_epochs: 2
    log_dirs: {'logs': './logs', 'weights': './weights'}
    loss_weights: {'generator': 0.0, 'feature_extractor': 0.0833, 'discriminator': 0.01}
    hr_valid_dir: div2k/DIV2K_train_HR/
    lr_valid_dir: div2k/DIV2K_train_LR_bicubic/X2/
    hr_train_dir: div2k/DIV2K_train_HR/
    lr_train_dir: div2k/DIV2K_train_LR_bicubic/X2/
    lr_patch_size: 50
    steps_per_epoch: 20
    batch_size: 4
    starting_epoch: 0
  generator: 
    name: rrdn
    parameters: {'C': 4, 'D': 3,

Epoch 0/2
Current learning rate: 0.00039999998989515007
100%|██████████| 20/20 [16:21<00:00, 49.06s/it]
Epoch 0 took      981.2s


200/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

{'val_loss': 0.6364007225632667, 'val_generator_loss': 0.3866208, 'val_discriminator_loss': 0.10027265, 'val_feature_extractor_loss': 2.9147494, 'val_feature_extractor_1_loss': 12.340904, 'val_generator_PSNR_Y': 8.907459, 'train_d_real_loss': 0.026073901, 'train_d_real_accuracy': 1.0, 'train_d_fake_loss': 0.23342907, 'train_d_fake_accuracy': 0.8808, 'train_loss': 0.64525557, 'train_generator_loss': 0.61285233, 'train_discriminator_loss': 0.076031156, 'train_feature_extractor_loss': 2.8233204, 'train_feature_extractor_1_loss': 12.650754, 'train_generator_PSNR_Y': 5.431891}
val_generator_PSNR_Y improved from       -inf to    8.90746
Saving weights
Epoch 1/2
Current learning rate: 0.00039999998989515007
100%|██████████| 20/20 [16:12<00:00, 48.62s/it]
Epoch 1 took      972.5s


200/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

{'val_loss': 0.5811756771802902, 'val_generator_loss': 0.2993842, 'val_discriminator_loss': 17.902983, 'val_feature_extractor_loss': 2.104007, 'val_feature_extractor_1_loss': 7.5513554, 'val_generator_PSNR_Y': 11.019596, 'train_d_real_loss': 0.26013958, 'train_d_real_accuracy': 0.9668, 'train_d_fake_loss': 0.7920469, 'train_d_fake_accuracy': 0.632, 'train_loss': 0.62647855, 'train_generator_loss': 0.34852964, 'train_discriminator_loss': 14.855622, 'train_feature_extractor_loss': 2.3733504, 'train_feature_extractor_1_loss': 9.101376, 'train_generator_PSNR_Y': 10.141146}
val_generator_PSNR_Y improved from    8.90746 to   11.01960
Saving weights
